In [57]:
from langchain_experimental.agents import create_csv_agent
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()
import requests
import pandas as pd

In [34]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"))

Here I converted the code into json so it is easy to embed them into the vector database

In [6]:
import csv
import json

csv_file = 'clean_data.csv'
json_file = 'data.json'

data = []
with open(csv_file, 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        data.append(row)

with open(json_file, 'w') as jsonfile:
    json.dump(data, jsonfile, indent=4)

In [4]:
agent1 = create_csv_agent(
            llm, "clean_data.csv", verbose=True)

In [5]:
agent1.run("i have about 100-200$ what is the best hotel i need 2 bedrooms, 2 bathrooms and a TV in the hall")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: The question is asking for the best hotel with specified requirements and a budget of 100-200$. However, the provided dataframe does not contain information about the number of bedrooms, bathrooms, or the presence of a TV in the hall in the hotels. It also does not have detailed price information, only a general 'price_range' column. Therefore, the information provided in the dataframe is not sufficient to answer the question accurately.
Final Answer: The information provided in the dataframe is not sufficient to accurately answer the question about the best hotel within a specified budget and with certain room requirements.

> Finished chain.


'The information provided in the dataframe is not sufficient to accurately answer the question about the best hotel within a specified budget and with certain room requirements.'

In [24]:
# import requests
# import time
# # Define the API endpoint and parameters
# url = "https://api-ares.traversaal.ai/live/predict"
# api_key = "ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a"
# headers = {"x-api-key": api_key, "content-type": "application/json"}
# query = "I want to eat best food san francisco, give me some suggestions"
# api_parameters = {"query": query}
# # Print out the request headers before sending the request
# print("Request Headers:")
# print(headers)
# # Retry the API request if a connection error occurs
# max_retries = 7
# retry_delay = 4
# #api_data={}
# for retry in range(max_retries):
#     try:
#         # Send a request to the API
#         api_response = requests.get(url, json=api_parameters, headers=headers)
#         # Extract the API response data
#         api_data = api_response.json()
#         break
#     except requests.exceptions.ConnectionError:
#         print(f"Connection error occurred. Retrying in {retry_delay} seconds...")
#         time.sleep(retry_delay)
# else:
#     print("Max retries exceeded. Unable to establish a connection to the API.")

# # Create the RAG prompt
# prompt = "I am performing RAG (Retrieve, Augment, Generate) using an API. Here is the context:\n\n"

# print(api_data)

# # # Iterate over the API data and generate responses for each query
# # for item in api_data["data"]:
# #     prompt += f"{item['context']}\n\n"
# #     prompt += f"Query: {item['query']}\n\n"
# #     prompt += "Generate a response using the provided context:\n\n"

# #     # Generate a response using the RAG model
# #     response = llm.invoke(prompt)
# #     # Extract the generated response
# #     generated_response = response

# #     # Print the generated response
# #     print(generated_response)

Request Headers:
{'x-api-key': 'ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a', 'content-type': 'application/json'}
{'message': 'Missing Authentication Token'}


In [74]:
def api_call(text):
  url = "https://api-ares.traversaal.ai/live/predict"

  payload = { "query": [text]}
  headers = {
    "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
    "content-type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)

  # here we will use the llm to summarize the response received from the ares api
  response_data = response.json()
  #print(response_data)
  try:
    response_text = response_data['data']['response_text']
    # Continue processing the data...
  except KeyError:
    print("Error: Unexpected response from the API. Please try again or contact the api owner.")
    # Optionally, you can log the error or perform other error handling actions.
  web_urls = response_data['data']['web_url']

  if len(response_text) > 10000:
    response_text = response_text[:8000]
    prompt = f"Summarize the following text in 500-100 0 words and jsut summarize what you see and do not add anythhing else: {response_text}"
    summary = llm.invoke(prompt)
    print(summary)
  else:
    summary = response_text

  result = "{} My list is: {}".format(response_text, web_urls)

# Convert the result to a string
  result_str = str(result)

  return result_str

In [75]:
print(api_call("cheap hotels in san francisco with 2 bedrooms and 2 bathrooms and a TV in the hall"))

- City Center Inn & Suites is located at 240 7th Street in SoMa, San Francisco.
- It is 0.9 miles from the center of San Francisco and the closest landmark is the Asian Art Museum.
- Check-in time at City Center Inn & Suites is 3:00 PM and check-out time is 11:00 AM.
- The hotel offers free Wi-Fi and free parking.
- City Center Inn & Suites is approximately 11.2 miles from San Francisco and 11.5 miles from Oakland.
- KAYAK searches for all room deals available at City Center Inn & Suites in San Francisco and compares them to find the best rate for your stay.
- The latest starting price for this hotel is $75 per night.
- If you're looking for other hotels, flights, or car rentals, you can search for thousands of options with KAYAK. My list is: ['https://sanfransiscosuites.com/2-bedroom-suites-in-san-francisco/', 'https://www.booking.com/hotel/us/san-francisco-inn.html', 'https://www.kayak.com/San-Francisco-Hotels-City-Center-Inn-Suites.20346.ksp', 'https://au.hotels.com/ho109830/city-ce

In [60]:
import requests

url = "https://api-ares.traversaal.ai/live/predict"

payload = { "query": ["best hotels in Paris"] ,
           "limit": 5}
headers = {
  "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
  "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [61]:
response_data = response.json()
print(len(response_data))

1


In [62]:
print(response_data)

{'data': {'response_text': "Here is a list of the best hotels in Paris:\n\n1. Hôtel Dame Des Arts: Located in the Quartier Latin, this hotel offers stylish rooms, a rooftop bar with 360-degree views, and a restaurant serving contemporary Mexican cuisine.\n\n2. Madame Rêve Hotel: Situated near Les Halles, this hotel offers stylish rooms with monument views, a rooftop restaurant, and a central location close to popular attractions.\n\n3. The Ritz Paris: A historic hotel with a modern soul, The Ritz Paris offers luxurious rooms, a flagship restaurant with a female head chef, and the famous Bar Hemingway.\n\n4. Molitor: This hotel, known for its clear swimming pools, offers a beach club atmosphere in the heart of Paris. The rooms are modern and feature pool-facing portholes.\n\n5. Le Pavillon de la Reine: Located in the Marais neighborhood, this hotel features a beautiful courtyard, elegant interiors mixing old-world touches with contemporary furnishings, and individually styled rooms and 

In [63]:
response_text = response_data['data']['response_text']
web_urls = response_data['data']['web_url']

In [56]:
print(response_text)

Here is a list of the best hotels in Paris:

1. Hôtel Dame Des Arts: Located in the Quartier Latin, this hotel offers stylish rooms, a rooftop bar with 360-degree views, and a restaurant serving contemporary Mexican cuisine.

2. Madame Rêve Hotel: Situated near Les Halles, this hotel offers stylish rooms with monument views, a rooftop restaurant, and a central location close to popular attractions.

3. The Ritz Paris: A historic hotel with a modern soul, The Ritz Paris offers luxurious rooms, a flagship restaurant with a female head chef, and the famous Bar Hemingway.

4. Molitor: This hotel, known for its clear swimming pools, offers a beach club atmosphere in the heart of Paris. The rooms are modern and feature pool-facing portholes.

5. Le Pavillon de la Reine: Located in the Marais neighborhood, this hotel features a beautiful courtyard, elegant interiors mixing old-world touches with contemporary furnishings, and individually styled rooms and suites.

6. InterContinental Paris Le 

In [52]:
print(web_urls)

['https://www.cntraveller.com/gallery/paris-hotels', 'https://www.cntraveler.com/gallery/best-hotels-in-paris', 'https://everydayparisian.com/where-to-stay-in-paris/', 'https://frenchly.us/best-hotels-paris/', 'https://thepointsguy.com/guide/best-hotels-in-paris/', 'https://community.ricksteves.com/travel-forum/france-reviews/hotel-with-view-of-eiffel-tower', 'https://www.reddit.com/r/ParisTravelGuide/comments/10moay1/where_to_stay_in_paris/', 'https://www.thehotelguru.com/best-hotels-in/france/paris', 'https://www.fodors.com/world/europe/france/paris/hotels', 'https://indagare.com/destination/paris/hotels']


In [54]:
prompt = ""
prompt += f"{response_text}\n\n"
prompt += f"Query: best hotels in paris\n\n"
prompt += "Generate a response using the provided context:\n\n"

# Generate a response using the RAG model
response = llm.invoke(prompt)
# Extract the generated response
generated_response = response

# Print the generated response
print(generated_response)

content="If you're looking for the best hotels in Paris, you have several incredible options. The Hôtel Dame Des Arts, located in the Quartier Latin features stylish rooms and a rooftop bar with panoramic views. If you prefer a historic hotel, The Ritz Paris offers luxurious rooms and a flagship restaurant. For a beach club atmosphere in the heart of Paris, consider Molitor, known for its clear swimming pools. If you're looking for a location near the Palais Garnier, the InterContinental Paris Le Grand offers grandeur and a must-visit café. \n\nFor a boutique experience, consider Le Grand Quartier in the 10th arrondissement or L'Hôtel, located in the historic literary Paris neighborhood. If you prefer a château-hotel, Château de Versailles Le Grand Contrôle offers rooms and suites with nature-inspired wallpapers and ornate moldings. \n\nFor an affordable and fun Art Deco hotel, consider Le Pigalle, featuring well-designed rooms and a rooftop lounge bar. Lutetia, after a spectacular res